In [31]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

df1 = pd.read_csv('spam.csv')
df1['class'] = df1['Category'].map({'spam': 1, 'ham': 0})
y = df1['class']
X = df1['Message']




In [29]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


In [34]:
tk = Tokenizer()
tk.fit_on_texts(X_train)
word_dict = tk.index_word
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

maxlen = 80
X_train_pad = sequence.pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = sequence.pad_sequences(X_test_seq, maxlen=maxlen)

In [40]:

word_dict[0] = '<PAD>'
word_dict[1] = '<START>'
word_dict[2] = '<UNK>'
for el in X_train_pad[0]:
    print(word_dict[el], end=' ')

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> reply <UNK> win £100 weekly where will the 2006 fifa world cup be held send stop <UNK> 87239 <UNK> end service 

In [45]:
maxf = 6000
RNNSpam = Sequential()
RNNSpam.add(Embedding(maxf, 8))
RNNSpam.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
RNNSpam.add(Dense(1, activation='sigmoid'))


In [46]:
RNNSpam.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 8)           8000      
                                                                 
 lstm_2 (LSTM)               (None, 16)                1600      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 9617 (37.57 KB)
Trainable params: 9617 (37.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
RNNSpam.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
RNNSpam.fit(X_train_pad, y_train,batch_size=64,epochs=10,validation_data=(X_train_pad, y_test))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_2/embedding_2/embedding_lookup defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\arjun\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\arjun\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\arjun\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\arjun\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\arjun\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\arjun\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\arjun\AppData\